### STEP 1 - import and setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from model_resnet import get_resnet18_cifar
from adversarial import fgsm_attack

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


### STEP 2 - dataset and dataloader

In [2]:
# CIFAR-10 è già diviso in train/test
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
testset  = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader  = DataLoader(testset, batch_size=100, shuffle=False)


100%|██████████| 170M/170M [00:47<00:00, 3.60MB/s] 


### STEP 3 - model and optimization

In [3]:
model = get_resnet18_cifar().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


### STEP 4 - adversarial training loop

In [4]:
epochs = 10
epsilon = 0.1

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(trainloader):
        images, labels = images.to(device), labels.to(device)

        # crea batch avversario
        adv_images = fgsm_attack(model, images.clone(), labels, epsilon)

        # mixa clean + adversarial
        mixed_inputs = torch.cat([images, adv_images])
        mixed_labels = torch.cat([labels, labels])

        optimizer.zero_grad()
        outputs = model(mixed_inputs)
        loss = criterion(outputs, mixed_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss:.4f}")


 21%|██        | 82/391 [24:51<1:33:41, 18.19s/it]


KeyboardInterrupt: 